In [13]:
import time

from concurrent.futures import ThreadPoolExecutor

import numpy as np
import scipy.stats as stats
import plotly_express as px
from ipywidgets import interact_manual

In [2]:
TIME_SCALE_FACTOR = 1000

In [3]:
class Request:
    def __init__(self, processing_time):
        self.processing_time = processing_time
        self.queue_start_time = 0
        self.queue_end_time = 0
        self.queue_time = 0
        
    def queue(self):
        self.queue_start_time = time.time()
    
    def dequeue(self):
        self.queue_end_time = time.time()
        self.queue_time = self.queue_end_time - self.queue_start_time
    
    def process(self):
        self.dequeue()
        time.sleep(self.processing_time / TIME_SCALE_FACTOR)

In [4]:
class RequestGenerator:
    def __init__(
        self,
        average_arrival_time,
        average_processing_time,
        arrival_distribution="poisson",
        processing_time_distribution="gaussian",
        num_requests=1000):

        self.num_requests = num_requests
        self.requests_generated = 0

        if arrival_distribution == "poisson":
            self.request_arrival_times = \
                stats.poisson.rvs(average_arrival_time, size=num_requests)
        elif arrival_distribution == "uniform":
            # assuming min arrival time to be 0.2
            # distribution spans [loc, loc + scale]
            self.request_arrival_times = \
                stats.uniform.rvs(loc=0.2, scale=2 * (average_arrival_time - 0.2), size=num_requests)
        elif arrival_distribution == "gaussian":
            # assuming min arrival time to be 0.2
            # this is modeled by assuming mu - 3 * sigma is equal to 0.2
            # the ressidual points are clipped
            self.request_arrival_times = \
                stats.norm.rvs(loc=average_arrival_time, scale=(average_arrival_time - 0.2) / 3, size=num_requests)
            self.request_arrival_times = np.clip(self.request_arrival_times, 0.2, np.inf)
        elif arrival_distribution == "constant":
            self.request_arrival_times = \
                [average_arrival_time for x in range(num_requests)]
        else:
            raise ValueError()

        if processing_time_distribution == "uniform":
            # assuming min processing time to be 0
            # distribution spans [loc, loc + scale]
            self.request_processing_times = \
                stats.uniform.rvs(loc=0, scale=2 * average_processing_time, size=num_requests)
        elif processing_time_distribution == "gaussian":
            # assuming min processing time to be 0
            # this is modeled by assuming mu - 3 * sigma is equal to 0
            # the ressidual points are clipped
            self.request_processing_times = \
                stats.norm.rvs(loc=average_processing_time, scale=average_processing_time / 3, size=num_requests)
            self.request_processing_times = np.clip(self.request_processing_times, 0, np.inf)
        elif processing_time_distribution == "constant":
            self.request_processing_times = \
                [average_processing_time for x in range(num_requests)]
        else:
            raise ValueError()
            
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.requests_generated < self.num_requests:
            time.sleep(self.request_arrival_times[self.requests_generated] / TIME_SCALE_FACTOR)
            request = Request(self.request_processing_times[self.requests_generated])
            self.requests_generated += 1
            return request
        else:
            raise StopIteration

In [5]:
class Server:
    def __init__(self, num_workers=1):
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        
    def process_request(self, request):
        self.executor.submit(lambda: request.process())
    
    def shutdown(self):
        self.executor.shutdown()

In [8]:
def visualize_wait_time(
        arrival_distribution="poisson",
        processing_time_distribution="gaussian"):

    avg_queuing_times = []

    for utilization_factor in np.linspace(0, 2, 10):
        average_arrival_time = 1
        average_processing_time = utilization_factor

        requests = []
        server = Server()
        for request in RequestGenerator(
                average_arrival_time,
                average_processing_time,
                arrival_distribution,
                processing_time_distribution):
            requests.append(request)
            request.queue()
            server.process_request(request)

        server.shutdown()
        avg_queuing_times.append((utilization_factor, np.mean([x.queue_time for x in requests])))

    px.line(x=[x for x, _ in avg_queuing_times], y=[y for _, y in avg_queuing_times]).show()

In [14]:
interact_manual(
    visualize_wait_time,
    arrival_distribution=["poisson", "uniform", "constant", "gaussian"],
    processing_time_distribution=["uniform", "constant", "gaussian"])

interactive(children=(Dropdown(description='arrival_distribution', options=('poisson', 'uniform', 'constant', …

<function __main__.visualize_wait_time(arrival_distribution='poisson', processing_time_distribution='gaussian')>